In [1]:


!rm 'OnlineRetail.csv.gz' -f
!wget https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz



--2018-06-04 08:52:27--  https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7483128 (7.1M) [application/octet-stream]
Saving to: ‘OnlineRetail.csv.gz’

100%[======================================>] 7,483,128   --.-K/s   in 0.1s    

2018-06-04 08:52:28 (51.4 MB/s) - ‘OnlineRetail.csv.gz’ saved [7483128/7483128]



In [2]:
loadRetailData = sc.textFile("OnlineRetail.csv.gz")

for row in loadRetailData.take(5):
    print row

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom


In [3]:
header = loadRetailData.first()

In [4]:
header

u'InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country'

In [9]:
loadRetailData = loadRetailData.filter(lambda line: line != header).map(lambda l: l.split(","))

In [10]:
loadRetailData.take(5)

[[u'536365',
  u'85123A',
  u'WHITE HANGING HEART T-LIGHT HOLDER',
  u'6',
  u'12/1/10 8:26',
  u'2.55',
  u'17850',
  u'United Kingdom'],
 [u'536365',
  u'71053',
  u'WHITE METAL LANTERN',
  u'6',
  u'12/1/10 8:26',
  u'3.39',
  u'17850',
  u'United Kingdom'],
 [u'536365',
  u'84406B',
  u'CREAM CUPID HEARTS COAT HANGER',
  u'8',
  u'12/1/10 8:26',
  u'2.75',
  u'17850',
  u'United Kingdom'],
 [u'536365',
  u'84029G',
  u'KNITTED UNION FLAG HOT WATER BOTTLE',
  u'6',
  u'12/1/10 8:26',
  u'3.39',
  u'17850',
  u'United Kingdom'],
 [u'536365',
  u'84029E',
  u'RED WOOLLY HOTTIE WHITE HEART.',
  u'6',
  u'12/1/10 8:26',
  u'3.39',
  u'17850',
  u'United Kingdom']]

In [11]:
for row in loadRetailData.take(5):
    print row

[u'536365', u'85123A', u'WHITE HANGING HEART T-LIGHT HOLDER', u'6', u'12/1/10 8:26', u'2.55', u'17850', u'United Kingdom']
[u'536365', u'71053', u'WHITE METAL LANTERN', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']
[u'536365', u'84406B', u'CREAM CUPID HEARTS COAT HANGER', u'8', u'12/1/10 8:26', u'2.75', u'17850', u'United Kingdom']
[u'536365', u'84029G', u'KNITTED UNION FLAG HOT WATER BOTTLE', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']
[u'536365', u'84029E', u'RED WOOLLY HOTTIE WHITE HEART.', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']


In [12]:
import re

loadRetailData = loadRetailData.filter(lambda l: int(l[3]) > 0\
                                and len(re.sub("\D", "", l[1])) != 0 \
                                and len(l[6]) != 0)

print loadRetailData.take(2)

[[u'536365', u'85123A', u'WHITE HANGING HEART T-LIGHT HOLDER', u'6', u'12/1/10 8:26', u'2.55', u'17850', u'United Kingdom'], [u'536365', u'71053', u'WHITE METAL LANTERN', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']]


In [14]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

loadRetailData = loadRetailData.map(lambda l: Row(inv=int(l[0]),\
                                    stockCode=int(re.sub("\D", "", l[1])),\
                                    description=l[2],\
                                    quant=int(l[3]),\
                                    invDate=l[4],\
                                    price=float(l[5]),\
                                    custId=int(l[6]),\
                                    country=l[7]))

In [17]:
retailDF = sqlContext.createDataFrame(loadRetailData)
print retailDF.printSchema()

root
 |-- country: string (nullable = true)
 |-- custId: long (nullable = true)
 |-- description: string (nullable = true)
 |-- inv: long (nullable = true)
 |-- invDate: string (nullable = true)
 |-- price: double (nullable = true)
 |-- quant: long (nullable = true)
 |-- stockCode: long (nullable = true)

None


In [18]:
retailDF.registerTempTable("retailPurchases")
sqlContext.sql("select * from retailPurchases limit 2").toPandas()

,country,custId,description,inv,invDate,price,quant,stockCode
0,United Kingdom,17850,WHITE HANGING HEART T-LIGHT HOLDER,536365,12/1/10 8:26,2.55,6,85123
1,United Kingdom,17850,WHITE METAL LANTERN,536365,12/1/10 8:26,3.39,6,71053


In [19]:
query = """
SELECT 
    custId, stockCode, 1 as purch
FROM 
    retailPurchases 
group 
    by custId, stockCode"""
retailDf = sqlContext.sql(query)
retailDf.registerTempTable("retailDf")

sqlContext.sql("select * from retailDf limit 10").toPandas()

,custId,stockCode,purch
0,18074,22224,1
1,13705,21889,1
2,15862,22441,1
3,15862,21592,1
4,12838,22739,1
5,12838,22149,1
6,14078,22548,1
7,14078,22423,1
8,12433,21977,1
9,14696,84360,1


In [28]:
testDf, cvDf, trainDf = retailDF.randomSplit([.1,.1,.8],1)

In [29]:
print "trainDf count: ", trainDf.count(), " example: "
for row in trainDf.take(2): print row
print

print "cvDf count: ", cvDf.count(), " example: "
for row in cvDf.take(2): print row
print

print "testDf count: ", testDf.count(), " example: "
for row in testDf.take(2): print row
print


trainDf count:  317013  example: 
Row(country=u'Australia', custId=12386, description=u'ASSORTED BOTTLE TOP  MAGNETS ', inv=537676, invDate=u'12/8/10 9:53', price=0.36, quant=120, stockCode=22915)
Row(country=u'Australia', custId=12386, description=u'IVORY GIANT GARDEN THERMOMETER', inv=537676, invDate=u'12/8/10 9:53', price=5.95, quant=12, stockCode=22926)

cvDf count:  39864  example: 
Row(country=u'Australia', custId=12386, description=u'20 DOLLY PEGS RETROSPOT', inv=537676, invDate=u'12/8/10 9:53', price=1.25, quant=24, stockCode=22567)
Row(country=u'Australia', custId=12386, description=u'PLASTERS IN TIN VINTAGE PAISLEY ', inv=537676, invDate=u'12/8/10 9:53', price=1.65, quant=12, stockCode=22557)

testDf count:  39626  example: 
Row(country=u'Australia', custId=12386, description=u'BIRTHDAY PARTY CORDON BARRIER TAPE', inv=537676, invDate=u'12/8/10 9:53', price=1.25, quant=24, stockCode=22953)
Row(country=u'Australia', custId=12386, description=u'PORCELAIN ROSE SMALL', inv=540280,